In [1]:
import sys
print(f'Interpreter dir: {sys.executable}')
import os

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('../')
    
print(f'Working dir: {os.getcwd()}')
%load_ext autoreload
%autoreload 2

Interpreter dir: /usr/bin/python3
Working dir: /home/guillem/waterhack/water_hackathlon


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold,cross_val_score,ShuffleSplit,StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix,classification_report

from scipy.stats import randint,uniform
import lightgbm as lgb

from waterhack.visualize import time_vs_y
from waterhack.utils import find_col
from waterhack.utils import series_to_supervised

In [3]:
%%file model_evaluation.py
import warnings

warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

import numpy as np
import pandas as pd
from bokeh.models import HoverTool
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.manifold.t_sne import TSNE
import umap
try:
    import holoviews as hv
    import hvplot.streamz
    import hvplot
    import hvplot.pandas
    from holoviews import streams
    import streamz
    from streamz.dataframe import DataFrame as StreamzDataFrame
except:
    pass


def safe_margin(val, low=True, pct: float = 0.05):
    low_pct, high_pct = 1 - pct, 1 + pct
    func = min if low else max
    return func(val * low_pct, val * high_pct)


def safe_bounds(array, pct: float = 0.05):
    low_x, high_x = array.min(), array.max()
    low_x = safe_margin(low_x, pct=pct)
    high_x = safe_margin(high_x, pct=pct, low=False)
    return low_x, high_x
def predict_grid(model, X):
    x_grid, y_grid = example_meshgrid(X)
    grid = np.c_[x_grid.ravel(), y_grid.ravel()]
    probs = model.predict_proba(grid)[:, 1].reshape(x_grid.shape)
    return probs, x_grid, y_grid
def plot_confussion_matrix(
    y_test,
    y_pred,
    target_names: list = None,
    cmap: str = "YlGnBu",
    width=500,
    height: int = 400,
    title: str = "Confusion matrix",
    normalize: bool = False,
):
    value_label = "examples"
    target_label = "true_label"
    pred_label = "predicted_label"
    label_color = "color"

    def melt_distances_to_heatmap(distances: pd.DataFrame) -> pd.DataFrame:
        dist_melt = pd.melt(distances.reset_index(), value_name=value_label, id_vars="index")
        dist_melt = dist_melt.rename(columns={"index": target_label, "variable": pred_label})
        dist_melt[target_label] = pd.Categorical(dist_melt[target_label])
        dist_melt[pred_label] = pd.Categorical(dist_melt[pred_label])
        coords = dist_melt.copy()
        coords[target_label] = dist_melt[target_label].values.codes
        coords[pred_label] = dist_melt[pred_label].values.codes
        return coords[[pred_label, target_label, value_label]]

    conf_matrix = confusion_matrix(y_test, y_pred)
    if normalize:
        conf_matrix = np.round(
            conf_matrix.astype("float") / conf_matrix.sum(axis=1)[:, np.newaxis], 3
        )
    # Adjust label color to make them readable when displayed on top of any colormap
    df = melt_distances_to_heatmap(pd.DataFrame(conf_matrix))
    mean = df[value_label].mean()
    df[label_color] = -df[value_label].apply(lambda x: int(x > mean))
    if target_names is not None:
        df[target_label] = df[target_label].apply(lambda x: target_names[x])
        df[pred_label] = df[pred_label].apply(lambda x: target_names[x])
    true_label_name = "Actual label"
    pred_label_name = "Predicted label"

    tooltip = [
        (true_label_name, "@{%s}" % target_label),
        (pred_label_name, "@{%s}" % pred_label),
        ("Examples", "@{%s}" % value_label),
    ]
    hover = HoverTool(tooltips=tooltip)
    heatmap = hv.HeatMap(df, kdims=[pred_label, target_label])
    heatmap.opts(title=title, colorbar=True, cmap=cmap, width=width, height=height, tools=[hover])
    labeled = heatmap * hv.Labels(heatmap).opts(text_color=label_color, cmap=cmap)
    return labeled.options(xlabel=pred_label_name, ylabel=true_label_name, invert_yaxis=True)


def __plot_decision_boundaries(X, y, y_pred, resolution: int = 100, embedding=None):
    if embedding is None:
        embedding = TSNE(n_components=2, random_state=160290).fit_transform(X)

    x_min, x_max = safe_bounds(embedding[:, 0])
    y_min, y_max = safe_bounds(embedding[:, 1])
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, resolution), np.linspace(y_min, y_max, resolution)
    )

    # approximate Voronoi tesselation on resolution x resolution grid using 1-NN
    background_model = KNeighborsClassifier(n_neighbors=1).fit(embedding, y_pred)
    voronoi_bg = background_model.predict(np.c_[xx.ravel(), yy.ravel()])
    voronoi_bg = voronoi_bg.reshape((resolution, resolution))

    mesh = hv.QuadMesh((xx, yy, voronoi_bg)).opts(cmap="viridis")
    points = hv.Scatter(
        {"x": embedding[:, 0], "y": embedding[:, 1], "pred": y_pred, "class": y},
        kdims=["x", "y"],
        vdims=["pred", "class"],
    )
    errors = y_pred != y
    failed_points = hv.Scatter(
        {"x": embedding[errors, 0], "y": embedding[errors, 1]}, kdims=["x", "y"]
    ).opts(color="red", size=5, alpha=0.9)

    points = points.opts(
        color="pred", cmap="viridis", line_color="grey", size=10, alpha=0.8, tools=["hover"]
    )
    plot = mesh * points * failed_points
    plot = plot.opts(
        xaxis=None, yaxis=None, width=500, height=450, title="Decision boundaries on TSNE"
    )
    return plot


def plot_decision_boundaries(
    X_train,
    y_train,
    y_pred_train,
    X_test,
    y_test,
    y_pred_test,
    resolution: int = 100,
    embedding=None,
):
    X = np.concatenate([X_train, X_test])
    y = np.concatenate([y_train, y_test])
    y_pred = np.concatenate([y_pred_train, y_pred_test])

    if embedding is None:
        try:
            embedding = umap.UMAP(n_components=2, random_state=160290).fit_transform(X)
        except:
            from sklearn.manifold import TSNE

            embedding = TSNE(n_components=2, random_state=160290).fit_transform(X)
    x_min, x_max = safe_bounds(embedding[:, 0])
    y_min, y_max = safe_bounds(embedding[:, 1])
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, resolution), np.linspace(y_min, y_max, resolution)
    )

    # approximate Voronoi tesselation on resolution x resolution grid using 1-NN
    background_model = KNeighborsClassifier(n_neighbors=1).fit(embedding, y_pred)
    voronoi_bg = background_model.predict(np.c_[xx.ravel(), yy.ravel()])
    voronoi_bg = voronoi_bg.reshape((resolution, resolution))

    mesh = hv.QuadMesh((xx, yy, voronoi_bg)).opts(cmap="viridis", alpha=0.6)
    points_train = hv.Scatter(
        {
            "x": embedding[: len(y_train), 0],
            "y": embedding[: len(y_train), 1],
            "pred": y_pred_train,
            "class": y_train,
        },
        kdims=["x", "y"],
        vdims=["pred", "class"],
    )
    points_test = hv.Scatter(
        {
            "x": embedding[len(y_train) :, 0],
            "y": embedding[len(y_train) :, 1],
            "pred": y_pred_test,
            "class": y_test,
        },
        kdims=["x", "y"],
        vdims=["pred", "class"],
    )
    errors = y_pred != y
    failed_points = hv.Scatter(
        {"x": embedding[errors, 0], "y": embedding[errors, 1]}, kdims=["x", "y"]
    ).opts(color="red", size=2, alpha=0.9)

    points_train = points_train.opts(
        color="class", cmap="viridis", line_color="grey", size=10, alpha=0.8, tools=["hover"]
    )
    points_test = points_test.opts(
        color="class",
        cmap="viridis",
        line_color="grey",
        size=10,
        alpha=0.8,
        tools=["hover"],
        marker="square",
    )
    plot = mesh * points_train * points_test * failed_points
    plot = plot.opts(xaxis=None, yaxis=None, width=500, height=450, title="Fronteras de decisión")
    return plot


def plot_classification_report(y, y_pred, **kwargs):
    report = classification_report(y, y_pred, output_dict=True, **kwargs)
    df = pd.DataFrame(report).applymap(lambda x: "{:.2f}".format(x))
    df = df.T.reset_index()
    return hv.Table(df).opts(title="Classification report")


def plot_feature_importances(model, target_names=None, feature_names=None, stacked: bool = False):
    n_target, n_features = model.coef_.shape
    ix = feature_names if feature_names is not None else list(range(n_features))
    cols = (
        target_names[:n_target]
        if target_names is not None
        else ["class_{}".format(i) for i in range(n_target)]
    )
    df = pd.DataFrame(index=ix, columns=cols, data=model.coef_.T)
    df.index.name = "Features"
    df.columns.name = "output_class"
    bar = df.hvplot.bar(legend=True, stacked=stacked, rot=75)
    bar = bar.opts(ylabel="Aggregated coefficients", title="Feature importances")
    return bar


def plot_model_evaluation(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    target_names=None,
    feature_names=None,
    normalize: bool = False,
    resolution: int = 100,
    stacked: bool = False,
    embedding: np.ndarray = None,
):
    import panel as pn

    y_pred_test = model.predict(X_test)
    metrics = plot_classification_report(y=y_test, y_pred=y_pred_test, target_names=target_names)
    conf_mat = plot_confussion_matrix(
        y_test=y_test, y_pred=y_pred_test, target_names=target_names, normalize=normalize
    )
    bounds = plot_decision_boundaries(
        X_train=X_train,
        y_train=y_train,
        y_pred_train=model.predict(X_train),
        X_test=X_test,
        y_test=y_test,
        y_pred_test=model.predict(X_test),
        resolution=resolution,
        embedding=embedding,
    )
    # features = plot_feature_importances(
    #     model=model, target_names=target_names, feature_names=feature_names, stacked=stacked
    # )
    gspec = pn.GridSpec(
        min_height=700, height=700, max_height=1200, min_width=750, max_width=1980, width=750
    )
    gspec[0, 0] = bounds
    gspec[1, 1] = metrics
    gspec[1, 0] = pn.pane.HTML(str(model), margin=0)
    gspec[0, 1] = conf_mat
    # gspec[3:5, :] = features
    return gspec

Overwriting model_evaluation.py


In [4]:
raw = pd.read_csv('data/processed/DMAVolumePressureWeather.csv', sep=",",parse_dates = ['Timestamp','DateRaised'])
print(raw.info())
raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943668 entries, 0 to 943667
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   DMA           943668 non-null  object        
 1   Timestamp     943668 non-null  datetime64[ns]
 2   PressureBar   943668 non-null  float64       
 3   m3Volume      943668 non-null  float64       
 4   DateRaised    962 non-null     datetime64[ns]
 5   LeakType      962 non-null     object        
 6   DMAName       962 non-null     object        
 7   is_leakage    943668 non-null  int64         
 8   maxtempC      943668 non-null  int64         
 9   mintempC      943668 non-null  int64         
 10  totalSnow_cm  943668 non-null  float64       
 11  sunHour       943668 non-null  float64       
 12  uvIndex       943668 non-null  int64         
 13  DewPointC     943668 non-null  int64         
 14  humidity      943668 non-null  int64         
 15  precipMM      943

,DMA,Timestamp,PressureBar,m3Volume,DateRaised,LeakType,DMAName,is_leakage,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,DewPointC,humidity,precipMM,pressure,tempC
0,NEWSEVMA,2016-12-01,2.011256,11.210,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4
1,NORW21MA,2016-12-01,2.345946,8.094,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4
2,NEWSTUMA,2016-12-01,2.564902,0.000,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4
3,BURYRDMA,2016-12-01,2.208337,2.650,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4
4,NORW37MA,2016-12-01,2.514855,2.791,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4


In [5]:
selected_DMA = "NEWSEVMA"
df_filt = raw[raw.DMA == selected_DMA]
df_filt

,DMA,Timestamp,PressureBar,m3Volume,DateRaised,LeakType,DMAName,is_leakage,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,DewPointC,humidity,precipMM,pressure,tempC
0,NEWSEVMA,2016-12-01 00:00:00,2.011256,11.210000,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4
10,NEWSEVMA,2016-12-01 00:15:00,1.986232,10.410000,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4
16,NEWSEVMA,2016-12-01 00:30:00,2.051919,10.100000,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1031,4
21,NEWSEVMA,2016-12-01 00:45:00,2.073815,9.970000,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1030,3
27,NEWSEVMA,2016-12-01 01:00:00,1.979976,9.370000,NaT,NaN,NaN,0,7,3,0.0,6.8,2,3,95,0.0,1030,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943613,NEWSEVMA,2020-05-31 23:00:00,2.110763,9.030000,NaT,NaN,NaN,0,19,10,0.0,16.7,5,9,94,0.0,1026,10
943634,NEWSEVMA,2020-05-31 23:15:00,2.113888,8.569999,NaT,NaN,NaN,0,19,10,0.0,16.7,5,9,94,0.0,1026,10
943641,NEWSEVMA,2020-05-31 23:30:00,2.057624,8.640000,NaT,NaN,NaN,0,19,10,0.0,16.7,5,9,94,0.0,1026,10
943648,NEWSEVMA,2020-05-31 23:45:00,2.048247,8.180000,NaT,NaN,NaN,0,18,10,0.0,16.7,5,10,95,0.0,1026,10


In [6]:
df_filt[["is_leakage", "PressureBar", "m3Volume", "Timestamp"]].to_csv("data_dma.csv")

In [7]:
def add_time_features(x):
    x["day"] = x["Timestamp"].map(lambda x:x.day)
    x["month"] = x["Timestamp"].map(lambda x:x.month)
    x["weekday"] = x["Timestamp"].map(lambda x:x.weekday())
    return x.drop("Timestamp", axis=1)

In [8]:
times = add_time_features(df_filt[["Timestamp"]].copy())

In [9]:
columns = ["is_leakage", "PressureBar", "m3Volume"]
df = df_filt[columns].copy()

In [11]:
from sklearn.preprocessing import StandardScaler
# Mark outliers to be filtered
outlier_scaler = StandardScaler().fit(df.values[:, 1:])
no_outliers = (np.abs(outlier_scaler.transform(df.values[:, 1:])) < 2).all(1)
# Scaler for data with no outliers
feature_scaler = StandardScaler().fit(df.values[no_outliers, 1:])

In [12]:
clean_data = df_filt[["is_leakage", "PressureBar", "m3Volume", "Timestamp"]].set_index("Timestamp")[no_outliers]

In [13]:
def is_outlier(x):
    val = outlier_scaler.transform(x)
    return (np.abs(val).flatten() > 2).any()

def create_examples(input_data, times, tw, ignore_leaks=False):
    inout_seq = []
    inputs = []
    outputs = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw, 1:]
        train_label = int(input_data[i+tw - 6 * 4:i+tw, 0].flatten().any())
        #train_label = int(input_data[i+int(tw / 4):i+tw, 0].flatten().any())
        #mprof = create_matrix_profile(train_seq, 24)
        times_seq = times[i:i+tw]
        if is_outlier(train_seq):
            continue
        
        inputs.append(np.concatenate([#mprof,
                                     feature_scaler.transform(train_seq).flatten(),
                                     times_seq.flatten()]))
        outputs.append(train_label)
    return np.array(inputs), np.array(outputs)

In [16]:
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.datasets import make_imbalance
from imblearn.under_sampling import NearMiss
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold,cross_val_score,ShuffleSplit,StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix,classification_report
import lightgbm as lgb

from sklearn.svm import LinearSVC

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from collections import Counter

In [17]:
x, y = create_examples(df.values, times.values, 48*2)

In [18]:
np.save("data/processed/features", x.astype(np.float32))
np.save("data/processed/targets", y.astype(np.int64))

In [ ]:
import pickle
with open("model_series_12h.pck", "rb") as f:
    model = pickle.load(f)

In [ ]:
gbm_2 = lgb.LGBMClassifier(objective='binary',metric='binary',bagging_fraction=0.5,
                         categorical_features =cats,
                         n_estimators=150,
                         is_unbalance=True)

In [ ]:
gbm = lgb.LGBMClassifier(objective='binary',metric='binary',bagging_fraction=0.5,
                         categorical_features =cats,
                         n_estimators=100,
                         is_unbalance=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, random_state=42)

print('Training target statistics: {}'.format(Counter(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test)))

# Create a pipeline
pipeline = make_pipeline(NearMiss(version=2), LogisticRegression(max_iter=200, class_weight='balanced'))

In [ ]:
gbm.fit(X_train, y_train)

# Classify and report the results
print(classification_report_imbalanced(y_test, gbm.predict(X_test)))

In [ ]:
gbm_2.fit(X_train, y_train)

# Classify and report the results
print(classification_report_imbalanced(y_test, gbm_2.predict(X_test)))

In [ ]:
y_train_pred = gbm_2.predict(X_train)
y_test_pred = gbm_2.predict(X_test)

print("TRAIN dataset")
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("TEST DATSET")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

In [ ]:
p = 0.46
y_train_pred = (model.predict_proba(X_train)[:, 1] > p).astype(int)
y_test_pred = (model.predict_proba(X_test)[:, 1] > p).astype(int)

print("TRAIN dataset")
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("TEST DATSET")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

In [ ]:
t1, t2 = 0.2, 0.55
y_train_pred = np.logical_and(gbm_2.predict_proba(X_train)[:, 1] > t1,
                              gbm.predict_proba(X_train)[:, 1] > t2).astype(int)
y_test_pred = np.logical_and(gbm_2.predict_proba(X_test)[:, 1] > t1,
                             gbm.predict_proba(X_test)[:, 1] > t2).astype(int)

print("TRAIN dataset")
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("TEST DATSET")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

In [ ]:
# Define parameters
params = {
        'n_estimators': randint(100, 1200),
        }
# Split training set into validation
#cv_ss = TimeSeriesSplit(n_splits=10).split(X_train, y_train)
cv_ss = StratifiedKFold(n_splits=5).split(X_train, y_train)
# Define model
n_iter = 20 # Max numero de iteraciones
random_search = RandomizedSearchCV(gbm, param_distributions=params,
                                   n_iter = n_iter, n_jobs=4,
                                   scoring = "f1_macro",
                                   cv = cv_ss, verbose=10, random_state=1234 )
# Fit xgb
random_search.fit(X_train, y_train)

In [ ]:
print('\n Best hyperparameters:')
print(random_search.best_params_)
gbm_model = random_search.best_estimator_
print('Best model:')
print(gbm_model)

# Collect GridSearch CV results. Get the best estimator.
df_results = pd.DataFrame(random_search.cv_results_)
err = df_results[df_results.rank_test_score == 1].filter(regex=("split\d.*")).values
display(err)

In [ ]:
import pickle
with open("model_matrices_12h.pck", "wb") as f:
    pickle.dump(gbm_model, f)

In [ ]:
with open("model_series_12h.pck", "rb") as f:
    pickle.load(f)

In [ ]:
y_train_pred = gbm_model.predict(X_train)
y_test_pred = gbm_model.predict(X_test)

print("TRAIN dataset")
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("TEST DATSET")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

In [ ]:
print('\n Best hyperparameters:')
print(random_search.best_params_)
gbm_model = random_search.best_estimator_
print('Best model:')
print(gbm_model)

# Collect GridSearch CV results. Get the best estimator.
df_results = pd.DataFrame(random_search.cv_results_)
err = df_results[df_results.rank_test_score == 1].filter(regex=("split\d.*")).values
display(err)

In [ ]:
y_train_pred = gbm_model.predict(X_train)
y_test_pred = gbm_model.predict(X_test)

print("TRAIN dataset")
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("TEST DATSET")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

In [ ]:
print('\n Best hyperparameters:')
print(random_search.best_params_)
gbm_model = random_search.best_estimator_
print('Best model:')
print(gbm_model)

# Collect GridSearch CV results. Get the best estimator.
df_results = pd.DataFrame(random_search.cv_results_)
err = df_results[df_results.rank_test_score == 1].filter(regex=("split\d.*")).values
display(err)

In [ ]:
y_train_pred = gbm_model.predict(X_train)
y_test_pred = gbm_model.predict(X_test)

print("TRAIN dataset")
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("TEST DATSET")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

In [ ]:
from sklearn.metrics import precision_score, recall_score
def val_recall(t1, t2):
    y_train_pred = np.logical_and(gbm_2.predict_proba(X_train)[:, 1] > t1,
                              gbm.predict_proba(X_train)[:, 1] > t2).astype(int)
    y_test_pred = np.logical_and(gbm_2.predict_proba(X_test)[:, 1] > t1,
                                 gbm.predict_proba(X_test)[:, 1] > t2).astype(int)
    return (precision_score(y_train, y_train_pred),
            precision_score(y_test, y_test_pred),
            recall_score(y_train, y_train_pred),
            recall_score(y_test, y_test_pred))



In [ ]:
xx,yy = np.meshgrid(np.arange(0.05, 0.95, 0.05), np.arange(0.5, 0.95, 0.05))

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
train_recs = []
eval_recs = []
train_precs = []
eval_precs = []
for t1, t2 in tqdm(list(zip(xx.ravel(), yy.ravel()))):
    tra, tes, ptra, ptes = val_recall(t1, t2)
    train_recs.append(tra)
    eval_recs.append(tes)
    train_precs.append(ptra)
    eval_precs.append(ptes)
    

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(xx, yy, np.array(eval_precs).reshape(xx.shape), rstride=1, cstride=1, cmap=cm.inferno)
ax.plot_surface(xx, yy, np.array(eval_recs).reshape(xx.shape), rstride=1, cstride=1, cmap=cm.viridis, alpha=0.9)
plt.xlabel("x")
ax.set_zlim(.1, 0.8)
plt.ylabel("y")
plt.show()

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(xx, yy, np.array(eval_precs).reshape(xx.shape), rstride=1, cstride=1, cmap=cm.viridis)
plt.show()

In [ ]:
import stumpy
import numpy as np
from numba import cuda

your_time_series = np.random.normal(size=(4 * 24))
window_size = 12  # Approximately, how many data points might be found in a pattern
#all_gpu_devices = [device.id for device in cuda.list_devices()]  # Get a list of all available GPU devices

matrix_profile = stumpy.stump(your_time_series, m=window_size)#, device_id=all_gpu_devices)

In [ ]:
import torch
import torch.nn as nn
class MyLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_layer_size=256, output_size=1, num_layers=2):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=num_layers)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(num_layers,1,self.hidden_layer_size),
                            torch.zeros(num_layers,1,self.hidden_layer_size))
        self.output_size = output_size
        self.num_layers = num_layers

    def forward(self, input_seq):
        time_steps, bs, n_feats = input_seq.shape
        x = input_seq.view(len(input_seq) ,bs, -1)
        #print(x.shape, input_seq.shape)
        lstm_out, self.hidden_cell = self.lstm(x, self.hidden_cell)
        #print(lstm_out.shape)
        lstm_out = lstm_out.transpose(1, 0).reshape(bs * time_steps, -1)
        #print("lstm", lstm_out.shape)
        predictions = self.linear(lstm_out).reshape(bs, time_steps, self.output_size)[:, -1, :]
        return predictions

In [ ]:
lstm = MyLSTM(output_size=5)

In [ ]:
lstm(torch.FloatTensor(seqs[0][0]).unsqueeze(1))

In [ ]:
torch.tensor(seqs[0][0]).unsqueeze(1).shape

In [ ]:
model = LSTM().to(DEVICE)
loss_function = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
from tqdm.autonotebook import tqdm, trange

In [ ]:
epochs = 1500
bs = 32

for i in trange(epochs):
    input_batch = []
    target_batch = []
    for j, (seq, labels) in enumerate(seqs):
        input_batch.append(seq.unsqueeze(1))
        target_batch.append(labels.unsqueeze(1))
        if len(input_batch) == bs:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(model.num_layers, bs, model.hidden_layer_size).to(DEVICE),
                            torch.zeros(model.num_layers, bs, model.hidden_layer_size).to(DEVICE))
            #print(seq.shape, labels.shape)
            inps = torch.cat(input_batch, 1)
            tgts = torch.cat(target_batch, 1)
            #print(inps.shape, tgts.shape, len(input_batch), j)
            y_pred = model(inps)
            #print(y_pred.shape, tgts[0, :, 1].shape)

            single_loss = loss_function(y_pred, tgts[0, :, 0].unsqueeze(1))
            single_loss.backward()
            optimizer.step()
            input_batch = []
            target_batch = []

    if True:#i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

In [ ]:
640 / 32 / 5

In [ ]:
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

In [ ]:
preds = []
targets = []
for seq, labels in tqdm(seqs[:10000]):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(2, 1, model.hidden_layer_size).to(DEVICE),
                        torch.zeros(2, 1, model.hidden_layer_size).to(DEVICE))
        #print(seq.shape, labels.shape)
        y_pred = model(seq.unsqueeze(1))
        #print(y_pred.shape, labels.shape)
        preds.append(y_pred)
        targets.append(labels)
        #single_loss = loss_function(y_pred, labels)
        #single_loss.backward()
        #optimizer.step()